In [1]:
from photometry import Spot, ImageData, AperturePhotometry
import numpy as np
import matplotlib.pyplot as plt

### ImageData Module

In [18]:
repo = "embargo_new"
collections = ["LSSTComCam/nightlyValidation"]
instrument='LSSTComCam'
exposure=2024121000100
dataId={"exposure":exposure,"detector":7}
datasetType='postISRCCD'
idata = ImageData(repo, collections[0], instrument, dataId, datasetType)

In [5]:
datasets = idata.get_datasets()
# can also access with idata.datasets

One dataset found


In [7]:
image = idata.get_image() #If no dataset passed, it will try to detect one and take it
# can also access with idata.image

One dataset found


In [ ]:
idata.img.metadata["OBSID"] #Verify it is the ref we want

In [11]:
#One can also pass a dataset one ref only
idata.dataId = {"exposure":2024121000103,"detector":7}
dataset_2 = idata.get_datasets()
image_2 = idata.get_image(dataset=dataset_2[0])

One dataset found


In [15]:
idata.img.metadata["OBSID"] #Verify it is the new ref

'CC_O_20241210_000103'

### Spot Module

In [19]:
sp = Spot(mask_size = 150) #We give the size of the mask pinhole, it remains an indicator only

In [16]:
# Load the image we want

In [20]:
idata = ImageData(repo, collections[0], instrument, dataId, datasetType)
idata.dataId = {"exposure":2024121000100,"detector":7}
image = idata.get_image()

One dataset found


In [21]:
#One can get the estimated area of the spot on the focal plane
sp.get_mask_size()
print(sp.mask_area_fp_px)

46584.59020688224


In [22]:
found_spot = sp.find_spot(idata.img.getImage(), threshold_adu=100,minarea=1000) #Need to pass an afw image*
print(found_spot)

[618 peaks, area=1896, centroid=(3549, 1051.5), 5759 peaks, area=69473, centroid=(1669.8, 2586.53)]


In [23]:
found_spot.sort(key=lambda x: x.getArea(), reverse=True)

In [9]:
iteratively_found_spot = sp.find_spot_iteratively(idata.img.getImage(), threshold_adu_min=10,threshold_adu_max=100000,minarea=1000)
print(iteratively_found_spot)

[4525 peaks, area=43120, centroid=(1666.13, 2592.75)]


In [15]:
sp.get_spot_information() # Get the information from the spot, if no spot passed, will use the last ones detected 
print(sp.centroid, sp.x, sp.y, sp.radius)

(3549, 1051.5) 3549.0 1051.5 24.566553364370574


In [17]:
#Be carefull here, if multiple spots, only take the first one
sp.get_spot_information(spot = found_spot[0])
print(sp.centroid, sp.x, sp.y, sp.radius)

(3549, 1051.5) 3549.0 1051.5 24.566553364370574


In [29]:
# We can use the previously determined area
minarea = int(sp.mask_area_fp_px*.9)
found_spot = sp.find_spot(idata.img.getImage(), threshold_adu=100,minarea=minarea)
print(found_spot)
sp.get_spot_information(spot = found_spot[0])
print(sp.centroid, sp.x, sp.y, sp.radius)

[5759 peaks, area=69473, centroid=(1669.8, 2586.53)]
(1669.8, 2586.5) 1669.7968131504326 2586.5290976350525 148.70757453084389


#### Known spots

In [31]:
x, y, radius = sp.x, sp.y, sp.radius #Using previously found spot

In [32]:
sp = Spot(x=x ,y=y , radius=radius) #We give the size of the mask pinhole, it remains an indicator only

### AperturePhotometry module

It is mandatory to use a spot and an image

In [2]:
sp = Spot(x=1670 ,y=2587 , radius=149)
dataId={"exposure":2024121000100,"detector":7}
idata = ImageData("embargo_new", "LSSTComCam/nightlyValidation", "LSSTComCam", dataId, "postISRCCD")
idata.get_image()
ap = AperturePhotometry(image=idata, spot=sp)

One dataset found


In [3]:
background_2d_thrshld = ap.get_2d_background_threshold()

In [4]:
bkg_aperture = ap.generate_aperture(centroid=ap.Spot.centroid, radius=400)
background_2d_ap = ap.get_2d_background_aperture(aperture = bkg_aperture)

In [5]:
mean_bkg = ap.get_mean_background()
print(mean_bkg)

5.139276


In [6]:
dark_exp_bkg = ap.get_dark_background()

One dataset found


In [28]:
substracted_background_image = ap.get_substracted_background_image(background = mean_bkg)

In [29]:
aperture = ap.generate_aperture(ap.Spot.centroid, radius=200)
photometry = ap.do_aperture_photometry(image=substracted_background_image, aperture=aperture)
print(photometry)

72940408.17241915


In [14]:
forced_photometry = ap.do_forced_aperture_photometry(centroid=ap.Spot.centroid, radius=200)
print(photometry)

72968438.49451017
